# Digital Health: Deidentification & Privacy
*Estimated Time: 1 hour*

Developers: Hikari Murayama & Ashley Quiterio

Welcome! In this Jupyter Notebooks, we will learn about sensitive research data with a focus on privacy and data de-identification. Specifically, we will explore how data can be re-identified if participants have characteristics that stand out. Then, investigate potential ethical questions around working with sensitive location research data.

<img src="images/data_security.jpeg" width=700 height=700 />

<center> <a href="https://medium.com/@michaelarleigh/data-re-identification-and-the-end-of-privacy-4cf7f4102612">Image Source</a> </center>

Some questions to consider when walking through this notebook are: 
1. What kind of risks will my participants face if their data is re-identified? 
2. Will data breaches affect some participants more than others? 
3. What are the rights of research participants when it comes to data collection?


## Table of Contents:
1. <a href='#section1'>Data</a>

    1.1 <a href='#section1a'>Step Data</a>
    
    1.2 <a href='#section1b'>GPS Data</a>
    

2. <a href='#section2'>Deidentifying Data</a>
    
    2.1. <a href='#section2a'>What can be used to identify an individual?</a>
    
    2.2. <a href='#section2b'>How does each individual vary from the rest?</a>
    
    2.3. <a href='#section2c'>How can we protect the privacy of participants whose locations are recorded?</a>


3. <a href='#section3'>GPS Analysis</a>

    3.1 <a href='#section3a'>Spatial Aggregation </a>

    3.2 <a href='#section3b'>Temporal Aggregation</a>
    
    
4. <a href='#section3'>Next Steps</a>
 


In [ ]:
# Import Libraries
import pandas as pd
import geopandas as gpd
import numpy as np

# Plotting & Mapping Libraries
import matplotlib.pyplot as plt
import contextily as cx

## 1. Data <a id='section1'></a>

Let us start by introducing the key datasets. For this analysis, you will be using data from the [**Diamante Study**](https://diamante.healthysms.org). This mobile health study revolves around a phone application that collects data on each user's daily step count. Participants occasionally receive text messages to encourage increasing physical activity. You'll work with two datasets from this study: a *step dataset* and a *gps dataset*. 

### 1.1 Step data <a id='section1a'></a>
This dataset contains information on the step data of 101 participants from the Diamante Study. Each row represents the step information for a specific participant on a specific day. There are 54 columns and 13,197 rows. 

In [ ]:
step_df = pd.read_csv("diamante_data_students_2_12_steps.csv")
subset_step_df = step_df[["Participant",'gender_is_male', 'gender_is_female', 'gender_is_other', 'age',
                          'week_steps', 'day_steps_0', "daily_goal", "weekly_goal"]]

grouped_step = subset_step_df.groupby("Participant").mean() 
grouped_step = grouped_step.rename(columns={'gender_is_male': "Male", 'gender_is_female': "Female", 
                                            "gender_is_other":"Other", "age": "Age", 
                                            "week_steps":"Steps per Week",
                                            "day_steps_0":"Steps per Day", "daily_goal": "Daily Goal",
                                            "weekly_goal": "Weekly Goal"})
grouped_step.head()

### 1.2 GPS data <a id='section1b'></a>

The Diamante study also collected geographic data from these participants. We have included one set of these GPS data for one participant, 
> Note: This dataset has gone through some modifications from its original form to protect privacy to be used within this module. As an extra precaution, please do not share this data with anyone else and only use on your personal computer. *Do not upload this data to any public sites or domains*

In [ ]:
location_gdf = gpd.read_file('example_gps_data.json')

# Coerce datetime variable to be in pandas recognized date time
location_gdf['datetime'] = pd.to_datetime(location_gdf['datetime'], infer_datetime_format=True, utc=True)

location_gdf.head()

## 2. Deidentifying Data <a id='section2'></a>

<u>Before proceeding, please read section 6, *Privacy and Security*, in the **Data Science Social Justice Toolkit**.</u>
<br>
<br>


The purpose of deidentifying data is to protect the individuals represented with data. This process is not uniform for every dataset with sensitive information. Some common techniques include adding noise to the data, creating synthetic data based on real people, and hiding identifiable features. 

Overall, this process brings into question what data should be collected during studies. How essential is all the information being collected for the sake of a study. Using digital health as a case study, we will explore how difficult it is to deidentify data, but still have something useful for analysis.

### 2.1: What can be used to identify an individual? <a id='section2a'></a>

Let's start by adding a new point to our original dataset. Recall your physical activity over the past few months. 

<div class="alert alert-info">
    
**Question 1:** What was your average step count on a given day? And, what was your daily step goal?

</div>

**WRITE YOUR ANSWER HERE. REPLACE THIS LINE WITH YOUR ANSWER BY DOUBLE-CLICKING THE CELL.**


How might this compare to others in the DIAMANTE study? Let's take a look into the study participants data. Below is a table and scatter plot comparing average steps per day against that individuals average daily goal.

In [ ]:
participant_data = grouped_step[['Steps per Day', 'Daily Goal']]
participant_data.plot.scatter("Daily Goal", 'Steps per Day')

participant_data.head()

<div class="alert alert-info">

**Question 2:** What patterns do you notice in the plot?
</div>

**WRITE YOUR ANSWER HERE.**


---
Now that we have an understanding for the individuals in the study, let's add your data to the plot.

In [ ]:
def add_point(daily_goal, number_steps):
    participant_data.plot.scatter("Daily Goal", 'Steps per Day')
    plt.scatter(daily_goal, number_steps, color='r');
    print("Your point was added with a daily goal of", daily_goal, "steps and", number_steps, "average number of steps.")

In [ ]:
# Run the add_point function 
add_point(1500, 10000)

Depending on the uniqueness of your reponse, your point is either hidden among others or an outlier in the scatterplot. 

<div class="alert alert-info">

**Question 3:** What might this mean for other individuals in the data? How identifiable are individuals who stray from the "norm" in visualizations? What risks might they face if their data is not deidentified?</div>


**WRITE YOUR ANSWER HERE.**

With only two features, there are only a few individuals who have a unique pair of average steps and step goals. However, how will this number increase if we include another feature (consider gender, age, race)?

In [ ]:
gender_cat = pd.Series(grouped_step[["Male", "Female", "Other"]].columns[np.where(grouped_step[["Male", "Female", "Other"]]!=0)[1]])
gender_cat

gender_col = pd.DataFrame(gender_cat)

with_gender = grouped_step[['Steps per Day', 'Daily Goal']].reset_index()
with_gender['Gender'] = gender_col

gender_nums = {0: {"Male": 'red', "Female": 'blue', 'Other': 'gold'}}
gender_col_1 = gender_col.replace(gender_nums)[0]

with_gender['Color by Gender'] = gender_col_1
with_gender.head()

In [ ]:
with_gender.plot.scatter("Daily Goal", 'Steps per Day', color=gender_col_1);

In [ ]:
# separate by gender into subplots 
fig = plt.figure(facecolor='white', figsize=(15,4));
ax = fig.add_subplot(131)
ax1 = fig.add_subplot(132)
ax2 = fig.add_subplot(133)

with_gender[with_gender['Gender'] =='Male'].plot.scatter("Daily Goal", 'Steps per Day', ax=ax, color='red');
with_gender[with_gender['Gender'] =='Female'].plot.scatter("Daily Goal", 'Steps per Day', ax=ax1, color='blue');
with_gender[with_gender['Gender'] =='Other'].plot.scatter("Daily Goal", 'Steps per Day', ax=ax2, color='gold');

ax.set_title('Male Participants');
ax1.set_title('Female Participants');
ax2.set_title("Other Gendered Participants");

In [ ]:
def add_point_gender(gender, daily_goal, number_steps):
#     Plots your daily goal by number of steps by gender across 3 scatterplots.
    fig = plt.figure(facecolor='white', figsize=(15,4));
    ax = fig.add_subplot(131)
    ax1 = fig.add_subplot(132)
    ax2 = fig.add_subplot(133)

    with_gender[with_gender['Gender'] =='Male'].plot.scatter("Daily Goal", 'Steps per Day', ax=ax, color='red');
    with_gender[with_gender['Gender'] =='Female'].plot.scatter("Daily Goal", 'Steps per Day', ax=ax1, color='blue');
    with_gender[with_gender['Gender'] =='Other'].plot.scatter("Daily Goal", 'Steps per Day', ax=ax2, color='gold');

    ax.set_title('Male Participants');
    ax1.set_title('Female Participants');
    ax2.set_title("Other Gendered Participants");
    
    if gender == 'Male':
        axis = ax
    elif gender =='Female':
        axis = ax1
    else: 
        axis = ax2
    
    axis.scatter(daily_goal, number_steps, color='cyan', marker='*');
    print("Your point was added to the", gender, "participants" ,"with a daily goal of", daily_goal, "steps and", number_steps, "average number of steps.")


In [ ]:
# Select from a gender category (Male, Female, Other) and add your Daily Goal and Steps per Day
add_point_gender("other", 15000, 10000)

The difference in unique features increases when there are fewer points (points are further organized by identifiable features).

### 2.2: How does each individual vary from the rest? <a id='section2b'></a>

We can compare individuals by investigating their impact on a distribution. To do this, we can use means and histograms. The following functions calculate the change in mean and distribution if we were to add a new `number_step` or `daily_goal` to the existing distributions.

In [ ]:
def comparison_by_steps(number_step):
    study_steps = np.array(grouped_step['Steps per Day'])
    original_mean = np.mean(study_steps)
    study_steps_modified = np.append(study_steps, number_step)
    modified_mean = np.mean(study_steps_modified)
    
    fig = plt.figure(facecolor='white', figsize=(15,4));
    ax = fig.add_subplot(121)
    ax1 = fig.add_subplot(122)
    bins = np.arange(0,19000,1000)
    ax.hist(study_steps, bins=bins, color='steelblue')
    ax.scatter(modified_mean, 1, color='black')  
    ax1.hist(study_steps_modified, bins=bins, color='dodgerblue')
    ax1.scatter(original_mean, 1, color='black')
    
    ax.set_title("Original Distribution")
    ax1.set_title("Modified Distribution")
    print("The original distribution had", round(original_mean,1),"for its average number of steps.")
    print("The modified distribution had", round(modified_mean,1),"for its average number of steps.")
    

In [ ]:
comparison_by_steps(10000)

In [ ]:
def comparison_by_daily_goal(daily_goal):
    study_goal = np.array(grouped_step['Daily Goal'])
    original_mean = np.mean(study_goal)
    study_goal_modified = np.append(study_goal, daily_goal)
    modified_mean = np.mean(study_goal_modified)
    
    fig = plt.figure(facecolor='white', figsize=(15,4));
    ax = fig.add_subplot(121)
    ax1 = fig.add_subplot(122)
    bins = np.arange(0,19000,1000)
    ax.hist(study_goal, bins=bins, color='steelblue')
    ax.scatter(modified_mean, 1, color='black')  
    ax1.hist(study_goal_modified, bins=bins, color='dodgerblue')
    ax1.scatter(original_mean, 1, color='black')
    
    ax.set_ylim([0,65])
    ax1.set_ylim([0,65])
    
    ax.set_xlim([0,18000])
    ax1.set_xlim([0,18000])
    
    ax.set_title("Original Distribution")
    ax1.set_title("Modified Distribution")
    print("The original distribution had", round(original_mean,1),"for its average daily goal.")
    print("The modified distribution had", round(modified_mean,1),"for its average daily goal.")
    

In [ ]:
comparison_by_daily_goal(1000)

<div class="alert alert-info">

**Question 4:** What is the impact of adding a new number of steps? How does that compare with adding a daily goal?</div>

**WRITE YOUR ANSWER HERE.**

### 2.3: How can we protect the privacy of participants whose locations are recorded? <a id='section2c'></a>

In the **Data Science Social Justice Toolkit**'s section on Privacy and Security, the case-study introduces Sarah and her circumstances surrounding her app data. 

<div class="alert alert-info">

**Question 5:** Can you think of another example where tracking location data may pose as a danger for users?</div>

**WRITE YOUR ANSWER HERE.**

Now, let's look at our GPS data for one of the participants and think about how we could preserve study participants' privacy in terms of their location. The GPS data reports the location of our participant at different times.

In [ ]:
location_gdf.head()

We have information that tells us the date and time of the location and the coordinate of the location.

Let's go ahead and check at what time intervals these coordinates are recorded.

In [ ]:
location_gdf['delta'] = (location_gdf['datetime']-location_gdf['datetime'].shift())
location_gdf['delta'].describe()

Assuming there is no interruption, this data was collected in 1 hour intervals.

We can go ahead and plot this to get an idea of what we're working with. We'll add a base plot to make sure we understand what types of locations we're looking at. Each blue dot represents a location that was recorded in the GPS dataset at a certain time.

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
location_gdf.to_crs('EPSG:3857').plot(ax=ax)
cx.add_basemap(ax)

## 3. GPS Analysis <a id='section3'></a>

### 3.1 Spatial aggregation <a id='section3a'></a>

As you can see, you can clearly see where this participant has gone and what areas they frequent. We're going to try *spatial aggregation* as one method to start protecting privacy. 

We'll try aggregating to the census block group level.

In [ ]:
# Read in census block groups
agg_shp_gdf = gpd.read_file('zip://indata/cb_2018_06_bg_500k.zip')

We're already created a function for you that aggregates for us. This function will find what census block group each location is in, and change that point to be the *centroid* of that polygon. The *centroid*, also known as the center of mass for your polygon, is a point where if you put the shape on the point of your finger it should be able to balance. For example, here is a map with the centroid of every state drawn.

<img src="http://giscommons.org/files/2009/08/3.17.gif">

We've created a `spatial_agg` function to do this aggregation for you.

In [ ]:
# Spatial aggregation
def spatial_agg(gdf, agg_shp_gdf,crs='EPSG:3857'):
    '''
    This function aggregates spatial locations to be the centroid of the aggregation shape.
    Potential aggregation levels include, but are not limited to, the census block or census block group level.
    Returns geodataframe with aggregated location.
    Points that do not lie within the aggregate shapes are excluded.
    
    Inputs:
    gdf: the geodataframe with point geometries meant to be spatially aggregated
    agg_shp: the geometries to be aggregated to as a geodataframe. 
    crs: user defined epsg code. default is 3857
    '''
    
    # Convert CRS to set one
    gdf_crs = gdf.to_crs(crs)
    agg_shp_gdf_crs = agg_shp_gdf.to_crs(crs)
    
    # spatial join
    sjoined = gpd.sjoin(agg_shp_gdf_crs, gdf_crs)
    
    # find centroid of polygon
    sjoined_centroid = sjoined['geometry'].centroid
    
    return sjoined_centroid

Let's try running this on our location data and aggregate to the census block group level.

In [ ]:
# Visualize with static mapping and then folium mapping
sjoined_centroid = spatial_agg(location_gdf,agg_shp_gdf)
sjoined_centroid.head()

We can then visualize our results to see how our points have shifted. The blue points are our original points, while the pink points are our spatially aggregated ones.

In [ ]:
xmin, xmax, ymin, ymax = location_gdf.total_bounds

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
agg_shp_gdf.to_crs('EPSG:3857').plot(color='lightgrey', ax=ax)
location_gdf.to_crs('EPSG:3857').plot(ax=ax)
sjoined_centroid.plot(color='pink', ax=ax)
ax.set_xlim(-1.364e7,-1.358e7)
ax.set_ylim(4.54e6,4.57e6)

And we can also plot this with our base map again to have a nice visual

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
location_gdf.to_crs('EPSG:3857').plot(ax=ax)
sjoined_centroid.plot(color='pink', ax=ax)
cx.add_basemap(ax)

<div class="alert alert-info">

**Question 6:** 
Update the above map to answer the following questions: 
- What kind of changes does spatial aggregation introduce? 
- What impacts on study results do you foresee with using this method?
    
</div>

**WRITE YOUR ANSWER HERE.**

### 3.2 Temporal aggregation <a id='section3b'></a>

Instead of aggregating in space, we can also aggregate our data over time. This means we can take the average location of a person for a specific time period.

As before, we've created a function for you that does this aggregation. We've included some flexibility to change the aggregation unit and number of units if you decide to use this function later.

In [ ]:
# Temporal aggregation
def temporal_agg(gdf,agg_time,dt_time_column = 'datetime', num=1):
    '''
    This function aggregates temporally, meaning locations will be averaged within a certain time frame.
    
    Inputs:
    gdf: the geodataframe with point geometries meant to be spatially aggregated
    dt_time_column: column name for date time
    agg_time: user specified unit of aggregation. possibilities: hour, day, month, year
    num: specify number of units to average over
    '''
    
    # Extract longitude and latitude
    gdf['x'] = gdf['geometry'].x
    gdf['y'] = gdf['geometry'].y
    
    # Grab different date time components
    gdf['year'] = gdf[dt_time_column].dt.year
    gdf['month'] = gdf[dt_time_column].dt.month
    gdf['day'] = gdf[dt_time_column].dt.day
    gdf['hour'] = gdf[dt_time_column].dt.hour
    
    # Create grouping scheme if larger than 1 time unit
    if num > 1:
        gdf[agg_time] = pd.cut(gdf[agg_time], np.arange(0,gdf[agg_time].max()+1, num))
    
    # Group data
    if agg_time =='year':
        agg_columns = ['year']
    elif agg_time =='month':
        agg_columns = ['year','month']
    elif agg_time =='day':
        agg_columns = ['year','month','day']
    elif agg_time =='hour':
        agg_columns = ['year','month','day','hour']
    
    time_agg = gdf[agg_columns + ['x','y']].groupby(agg_columns, as_index=False).mean()
    
    # Remove points we don't have data for
    time_agg  = time_agg[~time_agg.x.isna()]

    # Coerce back to a geodataframe
    time_agg_gdf = gpd.GeoDataFrame(time_agg, geometry=gpd.points_from_xy(time_agg.x, time_agg.y))
    
    time_agg_gdf.crs = gdf.crs

    return time_agg_gdf



Let's try taking a three hour average of our participant's location and see how that has changed.

In [ ]:
temporal_agg_gdf = temporal_agg(location_gdf,'hour',  dt_time_column = 'datetime', num=3)
temporal_agg_gdf.head()

Again, we'll visualize our results on top of our base map. This time our green points are those that have been temporally aggregated.

In [ ]:
# Visualize
fig, ax = plt.subplots(figsize=(15,15))
location_gdf.to_crs('EPSG:3857').plot(ax=ax)
temporal_agg_gdf.to_crs('EPSG:3857').plot(color='green', ax=ax)
cx.add_basemap(ax)

<div class="alert alert-info">

**Question 7:** How do the results of this temporal aggregation differ from the spatial aggregation? What are the advantages and disadvantages of each method?
</div>

**WRITE YOUR ANSWER HERE.**

As you may now see you could actually combine use different levels of aggregation for both methods, as well as combine temporal and spatial aggregation. This is the most basic level of aggregation and privacy preservation-- there are many more complex methods to go about this.

## 4. Next Steps <a id='section4'></a>

After formatting these plots, your task is to use these visualizations or others of your own creation in developing a policy brief, impact plan, and explainer video. Remember to continue using the **Data Science Social Justice Toolkit** as a resource. Here are some questions that build off of the analysis above that could be useful to scope your project:

For Step data:
- Imagine you were part of a digital health project. 
    - What question would you want to ask and what columns in our dataset would you like to use? 
    - Which features would you not originally collect? 
- What features do you feel are most identifiable? How does your expectation compare to a plot? Follow the steps from part 2 and create another scatterplot that shows the power of another feature.
- How would comparing distributions by gender be impacted by a new point?

For GPS data:
- *From a researcher perspective:* Recreate a statistic from Saeb et al and see how it differs by how these vary based on spatial or temporal aggregation? If you were a researcher, how would de-identification restrictions impact your results?
- *From a researcher perspective:* Instead of removing data via aggregation, we cab also add noise by adding additional random points (there is no right way to do this!). How does this influence your results? How does this influence a statistic from the Saeb et al paper? 
- *From a participant perspective*: Turn your location history "on" for Google maps for a week and extract your data ([here's a useful tutorial on how to do that]()). Bring this in into Python and visualize it. Then try aggregating at different spatial and temporal scales. How does your comfort level change with these different aggregation techniques? As a consumer of these mobile health applications, what input would you want to have to the researchers using your data? What types of policies would help protect your privacy?


Good luck!